# 03 · Pipeline Demo

We stitch together the manifesto pipeline: simulate β drift, run anomaly tests, and monitor estimators.
The goal is an audit dashboard, not a playbook for exploitation.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2

from wamecu import (
    BetaDriftConfig,
    beta_drift,
    ewma_estimator,
    kalman_tracker,
    stream_draws,
    chi_square_test,
    shannon_entropy,
    wamecu_probabilities,
)

plt.style.use('seaborn-v0_8')
RNG = np.random.default_rng(11)

## Simulate a monitoring scenario

We introduce a mid-run hardware nudge that amplifies β for outcomes 2 and 3.

In [ ]:
n_outcomes = 6
n_steps = 420
config = BetaDriftConfig(
    n_steps=n_steps,
    n_outcomes=n_outcomes,
    walk_scale=0.02,
    sin_amplitude=0.05,
    sin_period=120,
    clip=0.85,
    seed=99,
)
base_beta = beta_drift(config)
adjustment = np.zeros_like(base_beta)
adjustment[200:, 2:4] += 0.15
true_beta = np.clip(base_beta + adjustment, -0.9, 0.9)
observations = stream_draws(true_beta, seed=4242)
assert len(observations) == n_steps

## Diagnostic metrics

We compute chi-square and entropy deviations over a sliding window.

In [ ]:
window = 40
chi_stats = []
entropy_scores = []
support = np.arange(n_outcomes)
baseline_probs = np.full(n_outcomes, 1 / n_outcomes)
for t in range(window, n_steps + 1):
    window_obs = observations[t - window : t]
    counts = np.bincount(window_obs, minlength=n_outcomes)
    expected = baseline_probs * window
    stat, pval = chi_square_test(counts, expected)
    chi_stats.append({'step': t - 1, 'chi2_stat': stat, 'chi2_pvalue': pval})
    empirical_probs = counts / window
    entropy_scores.append({'step': t - 1, 'entropy': shannon_entropy(empirical_probs)})
chi_df = pd.DataFrame(chi_stats)
entropy_df = pd.DataFrame(entropy_scores)

## Permutation entropy (toy)

We approximate permutation entropy on the rolling draw stream to capture ordering anomalies.

In [ ]:
def permutation_entropy(series, order=3):
    series = np.asarray(series, dtype=float)
    if series.size < order:
        return np.nan
    patterns = {}
    for i in range(len(series) - order + 1):
        pattern = tuple(series[i : i + order].argsort())
        patterns[pattern] = patterns.get(pattern, 0) + 1
    counts = np.array(list(patterns.values()), dtype=float)
    probs = counts / counts.sum()
    mask = probs > 0
    return -np.sum(probs[mask] * np.log2(probs[mask]))

perm_window = 12
perm_records = []
for t in range(perm_window, n_steps + 1):
    segment = observations[t - perm_window : t]
    score = permutation_entropy(segment, order=3)
    perm_records.append({'step': t - 1, 'permutation_entropy': score})
perm_df = pd.DataFrame(perm_records)

## Adaptive estimators

We deploy the EWMA and Kalman trackers to recover β dynamics.

In [ ]:
ewma_trace = ewma_estimator(observations, n_outcomes=n_outcomes, alpha=0.07)
kalman_trace = kalman_tracker(observations, n_outcomes=n_outcomes, process_var=0.008, observation_var=0.08)

summary = pd.DataFrame({
    'step': np.arange(n_steps),
    'true_beta_2': true_beta[:, 2],
    'ewma_beta_2': ewma_trace[:, 2],
    'kalman_beta_2': kalman_trace[:, 2],
})
summary.head()

## Dashboard figure

Panels show (a) drift, (b) anomaly scores, and (c) estimator traces.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)
axes[0].plot(summary['step'], summary['true_beta_2'], color='black', label='True β₂', linewidth=2)
axes[0].set_ylabel('β₂')
axes[0].set_title('True drift for outcome 2')
axes[0].legend()

axes[1].plot(chi_df['step'], chi_df['chi2_stat'], label='χ² statistic')
axes[1].axhline(chi2.ppf(0.95, df=n_outcomes - 1), color='red', linestyle='--', label='95% threshold')
axes[1].set_ylabel('χ² stat')
axes[1].set_title('Chi-square anomaly score')
axes[1].legend()

axes[2].plot(summary['step'], summary['ewma_beta_2'], label='EWMA β₂')
axes[2].plot(summary['step'], summary['kalman_beta_2'], label='Kalman β₂')
axes[2].set_xlabel('Step')
axes[2].set_ylabel('β₂ estimate')
axes[2].set_title('Estimator traces')
axes[2].legend()

plt.tight_layout()
plt.show()

## Additional diagnostics

Permutation entropy dips alongside the hardware nudge, complementing χ² spikes.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(entropy_df['step'], entropy_df['entropy'], label='Shannon entropy', color='teal')
ax2 = ax.twinx()
ax2.plot(perm_df['step'], perm_df['permutation_entropy'], label='Permutation entropy', color='purple', alpha=0.6)
ax.set_xlabel('Step')
ax.set_ylabel('Shannon entropy (bits)')
ax2.set_ylabel('Permutation entropy (bits)')
ax.set_title('Entropy diagnostics over time')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout()
plt.show()

## Conclusion

* The anomaly stack (χ² + entropy) highlights the biased epoch clearly.
* Kalman reacts faster, while EWMA stabilises post-nudge—mirroring notebook 01.
* Future work: integrate quantum-drift hypotheses and entropy-leak cartography for cross-sensor validation.